<a href="https://colab.research.google.com/github/HubertasVin/deep-learning-practice/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importuojamos reikalingos bibliotekos

In [3]:
!pip install pyngrok
!pip install tabulate


import os
import numpy as np
import torch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


from google.colab import drive
drive.mount('/content/drive')

# Metrics
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

# API imports
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from io import BytesIO

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

DATA_DIR = '/content/drive/MyDrive/colab_content'
OI_DATA_DIR = DATA_DIR + "/OpenImages"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 60
LEARNING_RATE = 1e-4
TRAIN_MODE = True
NGROK_AUTH_TOKEN = "2v8l4mW8zOFneFzX7p47qORuEwS_4EcRU64N6iq1mUNEpXN3G"
API_ENABLE = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda:0


Duomenų įkėlimas ir paruošimas

In [4]:
def get_image_paths_and_labels(root_dir):
    classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    image_paths = []
    labels = []
    for cls in classes:
        images_dir = os.path.join(root_dir, cls, "images")
        if os.path.isdir(images_dir):
            for file in os.listdir(images_dir):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(images_dir, file))
                    labels.append(class_to_idx[cls])
        else:
            print(f"Warning: {images_dir} not found.")
    return image_paths, labels, classes, class_to_idx

image_paths, labels, classes, class_to_idx = get_image_paths_and_labels(OI_DATA_DIR)
print("Total images:", len(image_paths))
print("Classes found:", classes)

# Shuffle and split into training (80%) and validation (20%) sets
indices = np.arange(len(image_paths))
np.random.shuffle(indices)
split = int(0.8 * len(image_paths))
train_idx, val_idx = indices[:split], indices[split:]
train_paths = [image_paths[i] for i in train_idx]
train_labels = [labels[i] for i in train_idx]
val_paths   = [image_paths[i] for i in val_idx]
val_labels  = [labels[i] for i in val_idx]

# Image processing
def load_and_preprocess_image(path, label, training=True):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    if training:
        img = tf.image.random_flip_left_right(img)
    return img, label


train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(lambda p, l: load_and_preprocess_image(p, l, training=True),
                        num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(lambda p, l: load_and_preprocess_image(p, l, training=False),
                    num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Total images: 1061
Classes found: ['sandal', 'strawberry', 'traffic light']


Modelio kompiliavimas ir treniravimas

In [5]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')

model_path = os.path.join(DATA_DIR, "keras_model.h5")

if TRAIN_MODE:
    base_model.trainable = False

    inputs = keras.Input(shape=IMG_SIZE + (3,))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(classes), activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.summary()

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Phase 1 Training...")
    history1 = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print("Phase 2 Training...")
    history2 = model.fit(train_ds, epochs=20, validation_data=val_ds)

    # Save the model
    model.save(model_path)

    # Combine loss values from both phases for plotting
    total_epochs = len(history1.history['loss']) + len(history2.history['loss'])
    train_loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']

    # Plot the training history
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, total_epochs + 1), train_loss, label='Training Loss')
    plt.plot(range(1, total_epochs + 1), val_loss, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
else:
    print("Loading existing model...")
    model = keras.models.load_model(model_path)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide_1 (TrueDivide)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract_1 (Subtract)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,620,739 (10.00 MB)

 Trainable params: 361,987 (1.38 MB)

 Non-trainable params: 2,258,752 (8.62 MB)

Phase 1 Training...
Epoch 1/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.3288 - loss: 2.0203 - val_accuracy: 0.2958 - val_loss: 1.1162
Epoch 2/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 35s 219ms/step - accuracy: 0.3817 - loss: 1.7714 - val_accuracy: 0.2958 - val_loss: 1.1384
Epoch 3/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.4328 - loss: 1.6229 - val_accuracy: 0.2958 - val_loss: 1.1652
Epoch 4/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 219ms/step - accuracy: 0.4520 - loss: 1.5249 - val_accuracy: 0.2958 - val_loss: 1.1791
Epoch 5/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 144ms/step - accuracy: 0.4665 - loss: 1.5128 - val_accuracy: 0.2911 - val_loss: 1.1739
Epoch 6/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - accuracy: 0.4611 - loss: 1.5309 - val_accuracy: 0.3099 - val_loss: 1.1603
Epoch 7/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 19s 224ms/step - accuracy: 0.4792 - loss: 1.3784 - val_accuracy: 0.3615 - val_loss: 1.1395
Epoch 8/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - accuracy: 0.4699 - loss: 1.4

KeyboardInterrupt: 

Metrikos

In [ ]:
y_true = []
y_pred = []
for images, labels in val_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

cm = confusion_matrix(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print("\nValidation Metrics:")
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)

headers = [""] + classes
table = []
for i, row in enumerate(cm):
    table.append([classes[i]] + list(row))

print("\nConfusion Matrix:")
print(tabulate(table, headers=headers, tablefmt="grid"))

Pagalbinė prognozavimo funkcija

In [ ]:
def predict_image(image):
    # Resize and preprocess
    img = image.resize(IMG_SIZE)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    preds = model.predict(img)
    pred_class = np.argmax(preds, axis=1)[0]
    # index -> class name
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    return idx_to_class.get(pred_class, "Unknown")

API skirtas įvesti nuotraukas rankiniu būdu.

Nuotraukos testavimui su API:
- https://img.freepik.com/free-photo/yummy-strawberries-red-mellow-ripe-with-green-leafs-dark-desk_179666-391.jpg
- https://img.freepik.com/free-photo/strawberry-isolated-white-background_1232-1974.jpg
- https://img.freepik.com/free-photo/traffic-light-city-streets_23-2149091964.jpg
- https://img.freepik.com/free-photo/green-traffic-light-intersection_53876-153444.jpg
- https://img.freepik.com/premium-photo/close-up-yellow-street-traffic-light-hanging-from-pole_1048944-23361333.jpg
- https://img.freepik.com/free-photo/summer-slipper-white-shoes-sandals_1203-6528.jpg
- https://img.freepik.com/premium-photo/close-up-shoes-sand_1048944-30578092.jpg
- https://img.freepik.com/free-vector/man-brown-casual-flip-flop-sandal-shoes-vector_53876-25895.jpg

In [ ]:
app = Flask(__name__)

@app.route('/test', methods=['GET'])
def test_endpoint():
    return jsonify({"message": "Test endpoint working!"}), 200

@app.route('/predict', methods=['POST'])
def predict():
    if "file" in request.files:
        file = request.files["file"]
        try:
            image = tf.keras.preprocessing.image.load_img(file, target_size=IMG_SIZE)
        except Exception as e:
            return jsonify({"error": "Invalid image file."}), 400
    else:
        data = request.get_json(force=True)
        if "url" in data:
            response = requests.get(data["url"])
            image = tf.keras.preprocessing.image.load_img(BytesIO(response.content), target_size=IMG_SIZE)
        elif "file_path" in data:
            image = tf.keras.preprocessing.image.load_img(data["file_path"], target_size=IMG_SIZE)
        else:
            return jsonify({"error": "Provide 'url', 'file_path', or upload a file as 'file'."}), 400

    prediction = predict_image(image)
    return jsonify({"prediction": prediction})

if __name__ == '__main__':
    import sys
    # Start API server
    if API_ENABLE:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(5000)
        print("Public URL:", public_url)
        app.run(host='0.0.0.0', port=5000)